In [35]:
import numpy as np
import pandas as pd
import seaborn as sns

from mne import read_epochs
from mrmr import mrmr_classif
from ReliefF import ReliefF
from sklearn import svm
from sklearn.feature_selection import RFE
from random import randint

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import kurtosis, skew, moment, entropy, norm
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import  StratifiedKFold, cross_val_score, GridSearchCV
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('gruvboxd')

In [26]:
hbo = read_epochs('hbo_epo.fif')
hbr = read_epochs('hbr_epo.fif')

Reading C:\Users\Alberto\Documents\BCI\data\fd\hbo_epo.fif ...
    Found the data of interest:
        t =   -4992.00 ...   19968.00 ms
        0 CTF compensation matrices available
Not setting metadata
79 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\Alberto\Documents\BCI\data\fd\hbr_epo.fif ...
    Found the data of interest:
        t =   -4992.00 ...   19968.00 ms
        0 CTF compensation matrices available
Not setting metadata
79 matching events found
No baseline correction applied
0 projection items activated


In [27]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
rf = RandomForestClassifier(random_state=0)
svm_rbf = svm.SVC(kernel="rbf")
knn = KNeighborsClassifier(n_neighbors=3)
relief = ReliefF(n_neighbors=3, n_features_to_keep=6)
pca = PCA(n_components=.9)
le = LabelEncoder()

In [28]:
def rms(x):
    return np.sqrt(np.mean(x**2))

def slope(x):
    t = np.linspace(0, len(x)-1, len(x))
    return np.polyfit(t, x, 1)[0]

def autocorr(x):
    return float(np.correlate(x,x))
    
def temp_centroid(x):
    nom = [x[i] * i for i in range(x.shape[0])]
    return sum(nom)/sum(x)

def energy(x):
    return sum(x**2)

def med_abs_diff(x):
    return np.median(np.abs(np.diff(x)))

def mean_abs_diff(x):
    return np.mean(np.abs(np.diff(x)))

def calc_centroid(x, fs=7.81):
    energy = np.array(x) ** 2
    t = range(len(x))
    t = [float(x) / fs for x in t]
    t_energy = np.dot(np.array(t), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return centroid

left vs rest

In [43]:
conditions = ['left','rest']
hbo_sub = hbo[conditions].copy()
hbr_sub = hbr[conditions].copy()

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 0
t_max = 10

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()

In [44]:
#CAR
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

train_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

In [45]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

73.5017551181682
68.22289319045115


In [16]:
test_time_windows('lda','mrmr')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.17it/s]


0.7636363636363637  from 0  to  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.98it/s]


0.7872727272727273  from 0  to  7


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.11it/s]


0.8281818181818181  from 0  to  10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.98it/s]


0.8654545454545455  from 4  to  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11.85it/s]


In [46]:
test_classification('lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  7.78it/s]


pca 0.7090909090909091
rfe 0.7663636363636364
mrmr 0.8190909090909091
rf 0.6118181818181818


In [29]:
conditions = ['left','right']
hbo_sub = hbo[conditions].copy().apply_baseline((-5,0))
hbr_sub = hbr[conditions].copy().apply_baseline((-5,0))

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 0
t_max = 25

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()

Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)


In [30]:
#CAR
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

train_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

In [38]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

71.76973095348744
66.56471900648683


In [9]:
test_time_windows('lda','mrmr')

1 0


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.14it/s]


0.8290909090909093 std 0.06540402090401799  from 1  to  6
1 1


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.07it/s]


0.8400000000000001 std 0.08000000000000002  from 1  to  7
1 2


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 19.96it/s]


5 10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.85it/s]


5 11


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 17.06it/s]


In [31]:
selected_features, single_score = get_informative_features(1, 7)
selected_features = np.concatenate(selected_features)
np.save('single_fnirs_features', selected_features)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.82it/s]


mean accuracy:  0.7554545454545455 +- 0.12540512860721345


Rhythmn

In [32]:
conditions = ['r_pinch','r_stop']
hbo_sub = hbo[conditions].copy().apply_baseline((-5,0))
hbr_sub = hbr[conditions].copy().apply_baseline((-5,0))

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 0
t_max = 25

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()

#CAR
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

train_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)


In [11]:
test_time_windows('lda','mrmr', shift=5)

1 0


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.90it/s]


0.5199999999999999 std 0.08016511885472218  from 6  to  11
1 1


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 23.26it/s]


0.7254545454545456 std 0.038740501368477336  from 6  to  12
1 2


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 21.69it/s]


5 10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 21.55it/s]


5 11


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 21.37it/s]


In [33]:
selected_features, rhytmn_score = get_informative_features(6, 18)
selected_features = np.concatenate(selected_features)
np.save('rhy_fnirs_features', selected_features)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.80it/s]


mean accuracy:  0.7863636363636364 +- 0.1447283005902285


In [34]:
np.save('nirs_scores', np.array([single_score, rhytmn_score]))

3 class

In [7]:
conditions = ['left','rest','right']
hbo_sub = hbo[conditions].copy()
hbr_sub = hbr[conditions].copy()

hbo_sub.drop_channels(hbo_sub.info['bads'])
hbr_sub.drop_channels(hbr_sub.info['bads'])
y = le.fit_transform(hbr_sub.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

t_min = 0
t_max = 20

hbo_sub = hbo_sub.copy().crop(t_min,t_max).get_data()
hbr_sub = hbr_sub.copy().crop(t_min,t_max).get_data()

#CAR
hbo_avg = np.mean(hbo_sub, axis=0, keepdims=True)
hbr_avg = np.mean(hbr_sub, axis=0, keepdims=True)

hbo_sub = hbo_sub - hbo_avg
hbr_sub = hbr_sub - hbr_avg

train_data = np.concatenate([hbo_sub,  hbr_sub], axis=1)

In [12]:
test_time_windows('lda','mrmr')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.29it/s]


0.5650000000000001  from 0  to  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 18.52it/s]


0.5970833333333333  from 0  to  8


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 15.15it/s]


0.6224999999999999  from 2  to  8


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.27it/s]


0.6825  from 3  to  7


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 16.64it/s]


In [9]:
# compute conf level
def p(n):
    return (n*chance+2)/(n+4)
def conf(n,p,alpha):
    return np.sqrt((p*(1-p))/(n+4))*norm.ppf(1-(alpha/2))
def genConf(n, alpha):
    res = np.zeros((n))
    for i in range(0,n):
        res[i] = p(i) + conf(i, p(i), alpha)
    return res[-1]*100

In [8]:
def test_classification(classifier):
    pca_score = []
    rfe_score = []
    mrmr_score = []
    rf_score = []
    
    rfe_features = None
    mrmr_features = None
    estimator = None
    relief_fit = False
    
    for train_rep in range(10):
        cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=25 + train_rep)        
        cv_split = cv.split(train_data, y)

        pca_median_score = []
        rfe_median_score = []
        mrmr_median_score = []
        rf_median_score = []
        
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]
            
            x_train = []
            x_test = []
            
            x_train.append(np.apply_along_axis(np.std, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(np.mean, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(np.min, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(np.max, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(slope, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(skew, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(kurtosis, 2, train_data[train_idx]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, train_data[train_idx]))

#             x_train.append(np.apply_along_axis(energy, 2, train_data[train_idx]))
#             x_train.append(np.apply_along_axis(entropy, 2, train_data[train_idx]))
            
            x_test.append(np.apply_along_axis(np.std, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(np.mean, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(np.min, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(np.max, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(slope, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(skew, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(kurtosis, 2, train_data[test_idx]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, train_data[test_idx]))
            
            x_train = np.concatenate(x_train, axis=1)
            x_test = np.concatenate(x_test, axis=1)
            
            if classifier == 'svm':
                estimator = svm_rbf
            elif classifier == 'lda':
                estimator = lda
            elif classifier == 'rf':
                estimator = rf
            elif classifier == 'knn':
                estimator = knn
                
            pca       
            x_train_pca = pca.fit_transform(x_train.copy())
            x_test_pca = pca.transform(x_test.copy())  
            estimator.fit(x_train_pca, y_train)
            pca_median_score.append(np.median(estimator.score(x_test_pca, y_test)))

            # RFE
            if rfe_features is None:
                rfe = RFE(rf, n_features_to_select=10)
                rfe.fit(x_train, y_train)
                rfe_features = rfe.support_
            estimator.fit(x_train[:,rfe_features], y_train)
            rfe_median_score.append(np.median(estimator.score(x_test[:,rfe_features], y_test)))
            
            #mrmr
            if mrmr_features is None:            
                x_pd = pd.DataFrame(x_train)
                mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=10)
            all_f = np.linspace(0,x_train.shape[1],x_train.shape[1]).astype(int)
            selected = [True if i in mrmr_features else False for i in all_f]
           
            estimator.fit(x_train[:,mrmr_features], y_train)
            mrmr_median_score.append(np.median(estimator.score(x_test[:,mrmr_features], y_test)))
                
            #ReliefF
            if relief_fit == False:
                relief.fit(x_train, y_train)
                relief_fit = True
            x_train_rf = relief.transform(x_train.copy())
            x_test_rf = relief.transform(x_test.copy())
            estimator.fit(x_train_rf, y_train)
            rf_median_score.append(np.median(estimator.score(x_test_rf, y_test)))
                
        pca_score.append(np.median(pca_median_score))
        rfe_score.append(np.median(rfe_median_score))
        mrmr_score.append(np.median(mrmr_median_score))
        rf_score.append(np.median(rf_median_score))
        
    print('pca',np.mean(pca_score))
    print('rfe',np.mean(rfe_score))
    print('mrmr',np.mean(mrmr_score))
    print('rf',np.mean(rf_score))
    
def get_feature_names():
    all_features = []
    signal_features = ['std', 'mean', 'min', 'max', 'slope', 'skew', 'krts','tmp_centr']
    
    for ch in hbo.info['chs']:
        for ft in signal_features:
            all_features.append(ch['ch_name'] + ft)

    for ch in hbr.info['chs']:
        for ft in signal_features:
            all_features.append(ch['ch_name'] + ft)
                    
    return all_features 

def get_informative_features(start, end):
    score = []
    ft_names = get_feature_names() 
    selected_features = []

    t_0 = int((start)*7.81)
    t_1 = t_0 + int((end)*7.81)
    
    data = train_data[:,:,t_0:t_1] 

    for train_rep in range(10):  
        cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=randint(15,50) + train_rep)
        cv_split = cv.split(data, y)
        mrmr_features = None  
        median_score = []

        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]

            x_train = []
            x_test = []

            x_train.append(np.apply_along_axis(np.std, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(np.mean, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(np.min, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(np.max, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(slope, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(skew, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(kurtosis, 2, data[train_idx]))
            x_train.append(np.apply_along_axis(calc_centroid, 2, data[train_idx]))

            x_test.append(np.apply_along_axis(np.std, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(np.mean, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(np.min, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(np.max, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(slope, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(skew, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(kurtosis, 2, data[test_idx]))
            x_test.append(np.apply_along_axis(calc_centroid, 2, data[test_idx]))

            x_train = np.concatenate(x_train, axis=1)
            x_test = np.concatenate(x_test, axis=1)
            
            if mrmr_features is None:            
                x_pd = pd.DataFrame(x_train)
                mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=10)
                
            x_train =  x_train[:,mrmr_features]
            x_test = x_test[:,mrmr_features]

            lda.fit(x_train, y_train)
            median_score.append(np.median(lda.score(x_test, y_test)))
            
        selected_features.append(np.array(ft_names)[mrmr_features])
        score.append(np.median(median_score))
        
    print('mean accuracy: ',np.mean(score), '+-', np.std(score))
    return selected_features, np.mean(score)
                


In [1]:
def test_time_windows(classifier, selector,shift = 0):
    max_score = 0
    start = 1 
    while start < 6:
        step = 0 
        while step < 12:
            t_0 = int((start+shift)*7.81)
            t_1 = t_0 + int((step+shift+6)*7.81)
            print(start, step)
            data = train_data[:,:,t_0:t_1] 
            
            rfe_features = None
            relief_fit = False
            score = []
            for train_rep in range(5):
                cv = StratifiedKFold(n_splits=3,shuffle=True, random_state=25 + train_rep)
                cv_split = cv.split(data, y)
                
                median_score = []
       
                mrmr_features = None
    
                for train_idx, test_idx in cv_split:
                    y_train, y_test = y[train_idx], y[test_idx]

                    x_train = []
                    x_test = []

                    x_train.append(np.apply_along_axis(np.std, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(np.mean, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(np.min, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(np.max, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(slope, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(skew, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(kurtosis, 2, data[train_idx]))
                    x_train.append(np.apply_along_axis(calc_centroid, 2, data[train_idx]))

                    x_test.append(np.apply_along_axis(np.std, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(np.mean, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(np.min, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(np.max, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(slope, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(skew, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(kurtosis, 2, data[test_idx]))
                    x_test.append(np.apply_along_axis(calc_centroid, 2, data[test_idx]))

                    x_train = np.concatenate(x_train, axis=1)
                    x_test = np.concatenate(x_test, axis=1)
                    
                    if classifier == 'svm':
                        estimator = svm_rbf
                    elif classifier == 'lda':
                        estimator = lda
                    elif classifier == 'rf':
                        estimator = rf
                    elif classifier == 'knn':
                        estimator = knn
                        
                    if selector == 'rfe':
                        if rfe_features is None:
                            rfe = RFE(rf, n_features_to_select=10)
                            rfe.fit(x_train, y_train)
                            rfe_features = rfe.support_
                        
                        x_train =  x_train[:,rfe_features]
                        x_test = x_test[:,rfe_features]
                    elif selector == 'mrmr':
                        if mrmr_features is None:            
                            x_pd = pd.DataFrame(x_train)
                            mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=10)
                        all_f = np.linspace(0,x_train.shape[1],x_train.shape[1]).astype(int)
                        selected = [True if i in mrmr_features else False for i in all_f]

                        x_train =  x_train[:,mrmr_features]
                        x_test = x_test[:,mrmr_features]
                    else:
                        if relief_fit == False:
                            relief.fit(x_train, y_train)
                            relief_fit = True
                        x_train = relief.transform(x_train.copy())
                        x_test = relief.transform(x_test.copy())
                    
                    estimator.fit(x_train, y_train)
                    median_score.append(np.median(estimator.score(x_test, y_test)))
                
                score.append(np.median(median_score))
            
            if np.mean(score) > max_score:
                max_score = np.mean(score)
                print(max_score, 'std',  np.std(score), ' from', start + shift, ' to ', step+shift+6 )
        
            step +=1
        start += 1
                    